In [ ]:
def pull_building_osm(location):
    import osmnx as ox
    tags = {'building': 'yes'}
    place_name = location
    buildings = ox.geometries_from_place(place_name, tags)
    return buildings

In [ ]:
def slr_building(building, raster_path, name ):
    import rasterio
    from rasterstats import zonal_stats
    import pandas as pd
    import geopandas as gpd
    raster = rasterio.open(raster_path)
    #point = building['geometry'].representative_point()
    building_footprint = building.to_crs(crs=raster.crs.data)
    zs = zonal_stats(building_footprint, raster_path, stats=['max'])
    building_footprint = building_footprint.reset_index(drop=True)
    df1 = pd.DataFrame(zs)
    building_footprint = building_footprint.assign(name_column=df1['max'])
    building_footprint.rename(columns = {'name_column':f'{name}'}, inplace = True)
    building_non_na = building_footprint[building_footprint[f'{name}'].notna()]
    building_non_na = building_non_na.explode()
    building_non_na = building_non_na.to_crs("EPSG:4326")
    building_non_na = gpd.GeoDataFrame(building_non_na, geometry='geometry')
    return building_non_na

In [ ]:
la_building = pull_building_osm(location='Los Angeles County, California, United States of America')
la07_slr200_w100_path = r'LA07_flddepth_SLR200_W100.tif'
la07_slr200_w100 = slr_building(building=la_building, raster_path=la07_slr200_w100_path, name='slr200_w100').to_file('data/la06_slr200_w100.shp')